In [21]:
%matplotlib qt

import time

import matplotlib.pyplot as plt

import numpy as np
import exafmm.laplace as laplace

from fmm import Fmm
from fmm.kernel import laplace_p2p_serial, laplace_p2p_parallel

In [22]:
def timer(f, *args, **kwargs):
    s = time.time()
    res = f(*args, **kwargs)
    print(f'Runtime: {time.time()-s:.2f} s')
    return res

In [23]:
def sphere(npoints):
    
    phi = np.linspace(0, 2*np.pi, npoints)
    costheta = (np.linspace(0, 1, npoints)-0.5)*2

    theta = np.arccos(costheta)
    r = 0.5

    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)

    sources = np.vstack((x, y, z))

    return sources.T


In [31]:
test = sphere(100)

ax = plt.axes(projection='3d')
ax.scatter3D(test[:, 0], test[:, 1], test[:, 2], c='k', s=1)
plt.show()

In [5]:
exp = Fmm('test_config')
timer(exp.run)

Runtime: 0.94 s


In [6]:
# Set same sources, targets and source densities
src_coords = exp.sources
trg_coords = exp.targets
src_charges = exp.source_densities

In [7]:
# create a list of source instances
sources = laplace.init_sources(src_coords, src_charges)

# create a list of target instances
targets = laplace.init_targets(trg_coords)

In [8]:
fmm = laplace.LaplaceFmm(p=5, ncrit=exp.config['max_points'], filename="test_file.dat")

In [9]:
tree = laplace.setup(sources, targets, fmm)

In [10]:
trg_values = timer(laplace.evaluate, tree, fmm)

Runtime: 0.38 s


In [11]:
exp.target_potentials

array([1109.8365, 1013.8969, 1137.1888, ..., 1033.253 , 1073.9423,
       1096.0267], dtype=float32)

In [12]:
trg_values[:, 0]

array([1110.05684806, 1013.58162298, 1137.50210119, ..., 1032.98995011,
       1074.04157202, 1096.24109527])

In [13]:
np.allclose(exp.target_potentials, trg_values[:, 0], rtol=0.002, atol=0)

True